# 📘 Task Explanation: Layer Normalization (LN) CUDA Kernels  
*(Forward with Warp-Level Reduction & Backward with Gradient Propagation)*

## 🎯 Objective
The objective of this task is to design **CUDA kernel skeletons (without solution)** for **Layer Normalization (LN)**, covering both:
- **LN forward pass** using **warp-level reduction**
- **LN backward pass** for **gradient propagation**

This task focuses on **kernel structure, parallel decomposition, and data flow**, rather than providing a completed implementation.

---

## 🧠 Background: What Is Layer Normalization?
Layer Normalization is widely used in modern neural networks, especially in:
- Transformers (LLMs)
- RNNs and sequence models
- MLP blocks

For an input vector \( x \in \mathbb{R}^D \), LN computes:
\[
\mu = \frac{1}{D}\sum_{i=1}^{D} x_i, \quad
\sigma^2 = \frac{1}{D}\sum_{i=1}^{D}(x_i - \mu)^2
\]
\[
y_i = \gamma \cdot \frac{x_i - \mu}{\sqrt{\sigma^2 + \epsilon}} + \beta
\]

LN is **reduction-heavy** and **memory-sensitive**, making it an ideal case for:
- Warp-level reductions
- Register reuse
- Careful kernel design

---

## 🧩 Part A — LN Forward Kernel (Warp-Level Reduction)

### Task
Design a CUDA kernel skeleton for the **LN forward pass**, where:
- Each block (or warp) processes one LN row (one feature vector)
- Mean and variance are computed using **warp-level reduction**
- Normalization and affine transform (`gamma`, `beta`) are applied

### Key Design Requirements
- Use **warp-level primitives** (e.g., shuffle-based reduction)
- Avoid block-wide synchronization when possible
- Minimize global memory traffic
- Structure the kernel so that:
  - Reduction → normalization → write-back stages are clear

### What You Should Think About
- Mapping rows to warps or blocks
- How partial sums are accumulated
- Where registers vs shared memory would be used
- Numerical stability considerations (but not implementation)

---

## 🧩 Part B — LN Backward Kernel (Gradient Propagation)

### Task
Design a CUDA kernel skeleton for the **LN backward pass**, computing gradients for:
- Input (`dx`)
- Scale (`dgamma`)
- Bias (`dbeta`)

### Key Design Requirements
- Clearly separate gradient paths:
  - Gradient through normalization
  - Gradient through mean and variance
- Use parallel reduction patterns where needed
- Ensure correct data dependencies between steps
- Maintain a clean structure that mirrors the math of LN backward

### What You Should Think About
- Which gradients require reductions across the feature dimension
- Which values from the forward pass must be reused
- How to organize the kernel to avoid race conditions
- Where warp-level vs block-level reduction might be required

---

## 🧪 Deliverables
You should produce:
1. A **CUDA kernel skeleton** for LN forward (warp-level reduction)
2. A **CUDA kernel skeleton** for LN backward (gradient propagation)
3. Clearly marked `TODO` sections indicating:
   - Reduction logic
   - Memory access patterns
   - Synchronization points (if any)

> ⚠️ No actual implementation logic is required — only the kernel structure.

---

## 🎓 What You Learn from This Task
By completing this task, you will understand:
- How LN maps to GPU parallelism
- Why warp-level reduction is well-suited for LN
- How forward and backward passes differ in data flow
- How real ML kernels are structured internally

---

## 🚀 Relevance to ML Systems
Layer Normalization is a **core operator** in:
- Transformer blocks
- FlashAttention pipelines
- Fused LN + residual kernels
- High-performance inference and training engines

Being able to design LN kernel skeletons means you can:
- Read and understand optimized kernels (e.g., in PyTorch, Triton)
- Reason about performance bottlenecks
- Communicate effectively as an **ML Systems / GPU Kernel Engineer**

---

## 🧠 Key Takeaway
> **This task trains you to think in terms of GPU kernel structure and data flow for a real, production-critical ML operator—without relying on a finished implementation.**


In [ ]:
!nvcc --version
!nvidia-smi

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2024 NVIDIA Corporation
Built on Thu_Jun__6_02:18:23_PDT_2024
Cuda compilation tools, release 12.5, V12.5.82
Build cuda_12.5.r12.5/compiler.34385749_0
Wed Jan 21 12:31:33 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   52C    P8       

In [ ]:
!apt-get update
!apt-get install -y cuda-toolkit-12-4

Get:1 https://cli.github.com/packages stable InRelease [3,917 B]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:5 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ Packages [83.8 kB]
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [2,302 kB]
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:8 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:9 https://cli.github.com/packages stable/main amd64 Packages [354 B]
Get:10 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:11 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,877 kB]
Get:12 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]
Get:13 http://security.ubuntu.com/ubuntu jammy-se

In [ ]:
%%writefile ln_warp_skeleton.cu
#include <cstdio>
#include <cstdlib>
#include <cmath>
#include <cstring>
#include <cuda_runtime.h>

#define CUDA_CHECK(call) do {                                   \
  cudaError_t err = (call);                                     \
  if (err != cudaSuccess) {                                     \
    fprintf(stderr, "CUDA error %s:%d: %s\n",                   \
            __FILE__, __LINE__, cudaGetErrorString(err));       \
    std::exit(EXIT_FAILURE);                                    \
  }                                                             \
} while(0)

static inline float frand01(unsigned int &state) {
  state = 1664525u * state + 1013904223u;
  return (state & 0x00FFFFFF) / float(0x01000000);
}

// ============================================================
// CPU reference: LayerNorm forward
// y = (x - mean) * inv_std * gamma + beta
// saves mean and inv_std per row
// ============================================================
static void ln_forward_cpu(const float* x, const float* gamma, const float* beta,
                           float* y, float* mean, float* inv_std,
                           int B, int D, float eps) {

  // mean value
  for (int b = 0; b < B; ++b) {
    const float* xb = x + b * D;
    float m = 0.0f;
    for (int i = 0; i < D; ++i) m += xb[i];
    m /= (float)D;


  //variance
    float v = 0.0f;
    for (int i = 0; i < D; ++i) {
      float d = xb[i] - m;
      v += d * d;
    }
    v /= (float)D;

    float inv = 1.0f / std::sqrt(v + eps);
    mean[b] = m;
    inv_std[b] = inv;

  //forward propagation
    float* yb = y + b * D;
    for (int i = 0; i < D; ++i) {
      float xn = (xb[i] - m) * inv;
      yb[i] = xn * gamma[i] + beta[i];
    }
  }
}

// ============================================================
// CPU reference: LayerNorm backward
// Given dout, x, gamma, mean, inv_std -> compute dx, dgamma, dbeta
// NOTE: This is reference only. You can keep it as-is.
// ============================================================
static void ln_backward_cpu(const float* x, const float* gamma,
                            const float* mean, const float* inv_std,
                            const float* dout,
                            float* dx, float* dgamma, float* dbeta,
                            int B, int D) {
  // dgamma, dbeta accumulation across batch
  for (int i = 0; i < D; ++i) { dgamma[i] = 0.0f; dbeta[i] = 0.0f; }

  // dx per element
  for (int b = 0; b < B; ++b) {
    const float* xb = x + b * D;
    const float* db = dout + b * D;
    float* dxb = dx + b * D;

    float m = mean[b];
    float inv = inv_std[b];

    // xhat, dY*gamma
    // dx formula for LN (per-row):
    // dx = (1/D) * inv * (D*dxhat - sum(dxhat) - xhat*sum(dxhat*xhat))
    // where dxhat = dout * gamma, xhat = (x - mean) * inv
    float sum_dxhat = 0.0f;
    float sum_dxhat_xhat = 0.0f;


    //summation of gi and gx
    for (int i = 0; i < D; ++i) {
      float xhat = (xb[i] - m) * inv;
      float dxhat = db[i] * gamma[i];
      sum_dxhat += dxhat;
      sum_dxhat_xhat += dxhat * xhat;

      dgamma[i] += db[i] * xhat;
      dbeta[i]  += db[i];
    }

    //back propagation
    for (int i = 0; i < D; ++i) {
      float xhat = (xb[i] - m) * inv;
      float dxhat = db[i] * gamma[i];
      float val = ( (float)D * dxhat - sum_dxhat - xhat * sum_dxhat_xhat );
      dxb[i] = (inv / (float)D) * val;
    }
  }
}

// ============================================================
// TODO: Warp-level reduction helper using __shfl_down_sync
// ============================================================
__device__ __forceinline__ float warpReduceSum(float v) {
  // TODO: implement with __shfl_down_sync
  for(int offset = 16; offset > 0; offset >>= 1){
    v += __shfl_down_sync(0xffffffff, v, offset);
  }

  return v;
}

// ============================================================
// TODO: LN forward CUDA kernel (warp reduce)
// Suggested mapping:
//  - One row per block or per warp (your choice)
//  - Compute mean and invstd using warp-level reductions
//  - Write y and save mean/invstd
// ============================================================
__global__ void ln_forward_warp(const float* __restrict__ x,
                               const float* __restrict__ gamma,
                               const float* __restrict__ beta,
                               float* __restrict__ y,
                               float* __restrict__ mean,
                               float* __restrict__ invstd,
                               int B, int D, float eps) {
  // TODO

  int row = blockIdx.x;
  int tid = threadIdx.x;
  int lane = tid & 31;
  int warp = tid >> 5; // consider why it is like this instead of tid << 5
  int num_warp = (blockDim.x + 31)/ 32;

  const float* xrow = x + (size_t)row * D;
  float* yrow = y + (size_t)row * D;

  // step 1: each thread acculates partial sum and sumq

  float sumx = 0.0f;
  float sumq = 0.0f;
  for (int i = tid; i < D; i += blockDim.x){
      float xi = xrow[i];
      sumx += xi;
      sumq += xi * xi;
  }

  //step2: reduce within each warp
  sumx = warpReduceSum(sumx);
  sumq = warpReduceSum(sumq);

  //step3: write warp partials to shared memory, then reduce again using warp 0
  __shared__ float warp_sums[32];
  __shared__ float warp_sumsq[32];

  if (lane == 0){
      warp_sums[warp] = sumx;
      warp_sumsq[warp] = sumq;
  }
  __syncthreads();

  //get block sum
  float block_sum = 0.f, block_sumsq = 0.f;
  if(warp == 0){
    block_sum = (lane < num_warp) ? warp_sums[lane]  : 0.f;
    block_sumsq = (lane < num_warp) ? warp_sumsq[lane] : 0.f;

    block_sum   = warpReduceSum(block_sum);
    block_sumsq = warpReduceSum(block_sumsq);
  }

  //step4: broadcast mean and invstd to all threads
  __shared__ float sh_mu, sh_inv;
  if (tid == 0){
      float mu = block_sum / float(D);
      float var = block_sumsq / (float)D - mu * mu;
      float inv = rsqrtf(var + eps);
      sh_mu = mu;
      sh_inv = inv;
      mean[row] = mu;
      invstd[row] = inv;
  }
  __syncthreads();


  //step5: get result
  float mu = sh_mu, inv = sh_inv;
  for(int i = tid; i < D; i += blockDim.x){
      float xi = xrow[i];
      float xhat = (xi - mu) * inv;
      yrow[i] = xhat * gamma[i] + beta[i];
  }
}

// ============================================================
// TODO: LN backward CUDA kernel (gradient propagation)
// Inputs: x, gamma, mean, inv_std, dout
// Outputs: dx, dgamma, dbeta
//
// Notes:
//  - dgamma/dbeta are reduced across batch (need atomics OR 2-pass reduction)
//  - This skeleton does NOT prescribe a final strategy; you choose.
// ============================================================
__global__ void ln_backward_warp(const float* __restrict__ x,
                                const float* __restrict__ gamma,
                                const float* __restrict__ mean,
                                const float* __restrict__ inv_std,
                                const float* __restrict__ dout,
                                float* __restrict__ dx,
                                float* __restrict__ dgamma,
                                float* __restrict__ dbeta,
                                int B, int D) {
  // TODO
  int row = blockIdx.x;
  int tid = threadIdx.x;
  int lane = tid & 31;
  int warp = tid >> 5;
  int num_warps = (blockDim.x + 31) / 32;

  const float* xrow  = x  + (size_t)row * D;
  const float* dyrow = dout + (size_t)row * D;
  float* dxrow       = dx + (size_t)row * D;

  float mu = mean[row];
  float inv = inv_std[row];

  // step1: Accumulate partial sums for s1 and s2 in FP32
  float s1 = 0.f;   // sum(g)
  float s2 = 0.f;   // sum(g * xhat)

  for(int i = tid; i < D; i+= blockDim.x){
      float xi = xrow[i];
      float dyi = dyrow[i];
      float gi  = dyi * gamma[i];
      float xhat = (xi - mu) * inv;
      s1 += gi;
      s2 += gi * xhat;
  }

  // step2: Warp reduce

  s1 = warpReduceSum(s1);
  s2 = warpReduceSum(s2);

  // step3: Warp partials -> shared, then reduce with warp 0
  __shared__ float warp_s1[32];
  __shared__ float warp_s2[32];

  if (lane == 0) {
    warp_s1[warp] = s1;
    warp_s2[warp] = s2;
  }
  __syncthreads();

  float block_s1 = 0.f;
  float block_s2 = 0.f;

  if (warp == 0) {
    block_s1 = (lane < num_warps) ? warp_s1[lane] : 0.f;
    block_s2 = (lane < num_warps) ? warp_s2[lane] : 0.f;
    block_s1 = warpReduceSum(block_s1);
    block_s2 = warpReduceSum(block_s2);
  }

  // step4: Broadcast block_s1/block_s2
  __shared__ float sh_s1, sh_s2;
  if (tid == 0) {
    sh_s1 = block_s1;
    sh_s2 = block_s2;
  }
  __syncthreads();

  float S1 = sh_s1;
  float S2 = sh_s2;

  //step5: Write dx

  float invD = 1.0f / (float)D;

  for (int i = tid; i < D; i += blockDim.x) {
    float xi  = xrow[i];
    float dyi = dyrow[i];
    float gi  = dyi * gamma[i];
    float xhat = (xi - mu) * inv;

    float dx_i = inv * (gi - S1 * invD - xhat * (S2 * invD));
    dxrow[i] = dx_i;
    atomicAdd(&dbeta[i],  dyi);
    atomicAdd(&dgamma[i], dyi * xhat);

  }

}

// ============================================================
// Utilities: correctness + benchmark
// ============================================================
static bool check_allclose(const float* a, const float* b, int n, float rtol, float atol,
                           const char* name) {
  for (int i = 0; i < n; ++i) {
    float av = a[i], bv = b[i];
    float diff = std::fabs(av - bv);
    float tol = atol + rtol * std::fabs(bv);
    if (diff > tol || std::isnan(av) || std::isnan(bv)) {
      printf("Mismatch %s at %d: got=%g ref=%g diff=%g tol=%g\n", name, i, av, bv, diff, tol);
      return false;
    }
  }
  return true;
}

template <typename Kernel, typename... Args>
static float bench_kernel_ms(Kernel k, dim3 grid, dim3 block,
                             int warmup, int iters, Args... args) {
  for (int i = 0; i < warmup; ++i) {
    k<<<grid, block>>>(args...);
  }
  CUDA_CHECK(cudaGetLastError());
  CUDA_CHECK(cudaDeviceSynchronize());

  cudaEvent_t start, stop;
  CUDA_CHECK(cudaEventCreate(&start));
  CUDA_CHECK(cudaEventCreate(&stop));

  CUDA_CHECK(cudaEventRecord(start));
  for (int i = 0; i < iters; ++i) {
    k<<<grid, block>>>(args...);
  }
  CUDA_CHECK(cudaEventRecord(stop));
  CUDA_CHECK(cudaGetLastError());
  CUDA_CHECK(cudaEventSynchronize(stop));

  float ms = 0.0f;
  CUDA_CHECK(cudaEventElapsedTime(&ms, start, stop));
  CUDA_CHECK(cudaEventDestroy(start));
  CUDA_CHECK(cudaEventDestroy(stop));
  return ms / iters;
}

int main(int argc, char** argv) {
  // Args: ./ln <B> <D> <iters>
  int B = (argc > 1) ? std::atoi(argv[1]) : 4096;
  int D = (argc > 2) ? std::atoi(argv[2]) : 1024;
  int iters = (argc > 3) ? std::atoi(argv[3]) : 200;
  float eps = 1e-5f;

  printf("LN Skeleton: B=%d D=%d iters=%d\n", B, D, iters);

  size_t bytes_x = (size_t)B * D * sizeof(float);
  size_t bytes_d = (size_t)D * sizeof(float);
  size_t bytes_b = (size_t)B * sizeof(float);

  // Host buffers
  float* hx = (float*)std::malloc(bytes_x);
  float* hgamma = (float*)std::malloc(bytes_d);
  float* hbeta  = (float*)std::malloc(bytes_d);
  float* hy_ref = (float*)std::malloc(bytes_x);
  float* hy_gpu = (float*)std::malloc(bytes_x);
  float* hmean_ref = (float*)std::malloc(bytes_b);
  float* hinv_ref  = (float*)std::malloc(bytes_b);
  float* hmean_gpu = (float*)std::malloc(bytes_b);
  float* hinv_gpu  = (float*)std::malloc(bytes_b);

  float* hdout = (float*)std::malloc(bytes_x);
  float* hdx_ref = (float*)std::malloc(bytes_x);
  float* hdx_gpu = (float*)std::malloc(bytes_x);
  float* hdg_ref = (float*)std::malloc(bytes_d);
  float* hdb_ref = (float*)std::malloc(bytes_d);
  float* hdg_gpu = (float*)std::malloc(bytes_d);
  float* hdb_gpu = (float*)std::malloc(bytes_d);

  if (!hx||!hgamma||!hbeta||!hy_ref||!hy_gpu||!hmean_ref||!hinv_ref||!hmean_gpu||!hinv_gpu||
      !hdout||!hdx_ref||!hdx_gpu||!hdg_ref||!hdb_ref||!hdg_gpu||!hdb_gpu) {
    fprintf(stderr, "Host malloc failed.\n");
    return 1;
  }

  // Init data
  unsigned int st = 123u;
  for (int i = 0; i < B * D; ++i) hx[i] = (frand01(st) - 0.5f) * 2.0f;
  for (int i = 0; i < D; ++i) {
    hgamma[i] = 0.5f + frand01(st);
    hbeta[i]  = (frand01(st) - 0.5f) * 0.1f;
  }
  for (int i = 0; i < B * D; ++i) hdout[i] = (frand01(st) - 0.5f) * 2.0f;

  // CPU refs
  ln_forward_cpu(hx, hgamma, hbeta, hy_ref, hmean_ref, hinv_ref, B, D, eps);
  ln_backward_cpu(hx, hgamma, hmean_ref, hinv_ref, hdout, hdx_ref, hdg_ref, hdb_ref, B, D);

  // Device buffers
  float *dx=nullptr, *dgamma=nullptr, *dbeta=nullptr;
  float *x=nullptr, *gamma=nullptr, *beta=nullptr, *y=nullptr, *mean=nullptr, *inv=nullptr, *dout=nullptr;
  CUDA_CHECK(cudaMalloc(&x, bytes_x));
  CUDA_CHECK(cudaMalloc(&gamma, bytes_d));
  CUDA_CHECK(cudaMalloc(&beta, bytes_d));
  CUDA_CHECK(cudaMalloc(&y, bytes_x));
  CUDA_CHECK(cudaMalloc(&mean, bytes_b));
  CUDA_CHECK(cudaMalloc(&inv, bytes_b));
  CUDA_CHECK(cudaMalloc(&dout, bytes_x));
  CUDA_CHECK(cudaMalloc(&dx, bytes_x));
  CUDA_CHECK(cudaMalloc(&dgamma, bytes_d));
  CUDA_CHECK(cudaMalloc(&dbeta, bytes_d));

  CUDA_CHECK(cudaMemcpy(x, hx, bytes_x, cudaMemcpyHostToDevice));
  CUDA_CHECK(cudaMemcpy(gamma, hgamma, bytes_d, cudaMemcpyHostToDevice));
  CUDA_CHECK(cudaMemcpy(beta, hbeta, bytes_d, cudaMemcpyHostToDevice));
  CUDA_CHECK(cudaMemcpy(dout, hdout, bytes_x, cudaMemcpyHostToDevice));

  CUDA_CHECK(cudaMemset(y, 0, bytes_x));
  CUDA_CHECK(cudaMemset(mean, 0, bytes_b));
  CUDA_CHECK(cudaMemset(inv, 0, bytes_b));
  CUDA_CHECK(cudaMemset(dx, 0, bytes_x));
  CUDA_CHECK(cudaMemset(dgamma, 0, bytes_d));
  CUDA_CHECK(cudaMemset(dbeta, 0, bytes_d));

  // ------------------------------------------------------------
  // TODO: Choose launch config(s)
  // Forward:
  // dim3 f_block(0, 0, 1); // TODO
  // dim3 f_grid(0, 0, 1);  // TODO
  // Backward:
  // dim3 b_block(0, 0, 1); // TODO
  // dim3 b_grid(0, 0, 1);  // TODO
  // ------------------------------------------------------------
  auto round_up_warp = [](int x) { return (x + 31) & ~31; };

  int f_threads = 256;
  if (D <= 128) f_threads = 128;
  f_threads = round_up_warp(f_threads);

  dim3 f_block(f_threads, 1, 1);
  dim3 f_grid(B, 1, 1);

  int b_threads = 256;
  if (D <= 128) b_threads = 128;
  b_threads = round_up_warp(b_threads);

  dim3 b_block(b_threads, 1, 1);
  dim3 b_grid(B, 1, 1);

  // ---------------------------
  // Correctness: forward
  // ---------------------------
  ln_forward_warp<<<f_grid, f_block>>>(x, gamma, beta, y, mean, inv, B, D, eps);
  CUDA_CHECK(cudaGetLastError());
  CUDA_CHECK(cudaDeviceSynchronize());

  CUDA_CHECK(cudaMemcpy(hy_gpu, y, bytes_x, cudaMemcpyDeviceToHost));
  CUDA_CHECK(cudaMemcpy(hmean_gpu, mean, bytes_b, cudaMemcpyDeviceToHost));
  CUDA_CHECK(cudaMemcpy(hinv_gpu, inv, bytes_b, cudaMemcpyDeviceToHost));

  bool ok_y = check_allclose(hy_gpu, hy_ref, B*D, 1e-3f, 1e-3f, "y");
  bool ok_m = check_allclose(hmean_gpu, hmean_ref, B,   1e-4f, 1e-4f, "mean");
  bool ok_i = check_allclose(hinv_gpu,  hinv_ref,  B,   1e-4f, 1e-4f, "inv_std");
  printf("Forward correctness: %s\n", (ok_y && ok_m && ok_i) ? "PASS" : "FAIL");

  // ---------------------------
  // Correctness: backward
  // ---------------------------
  CUDA_CHECK(cudaMemset(dx, 0, bytes_x));
  CUDA_CHECK(cudaMemset(dgamma, 0, bytes_d));
  CUDA_CHECK(cudaMemset(dbeta, 0, bytes_d));

  ln_backward_warp<<<b_grid, b_block>>>(x, gamma, mean, inv, dout, dx, dgamma, dbeta, B, D);
  CUDA_CHECK(cudaGetLastError());
  CUDA_CHECK(cudaDeviceSynchronize());

  CUDA_CHECK(cudaMemcpy(hdx_gpu, dx, bytes_x, cudaMemcpyDeviceToHost));
  CUDA_CHECK(cudaMemcpy(hdg_gpu, dgamma, bytes_d, cudaMemcpyDeviceToHost));
  CUDA_CHECK(cudaMemcpy(hdb_gpu, dbeta, bytes_d, cudaMemcpyDeviceToHost));

  bool ok_dx = check_allclose(hdx_gpu, hdx_ref, B*D, 1e-2f, 1e-2f, "dx");
  bool ok_dg = check_allclose(hdg_gpu, hdg_ref, D,   1e-2f, 1e-2f, "dgamma");
  bool ok_db = check_allclose(hdb_gpu, hdb_ref, D,   1e-2f, 1e-2f, "dbeta");
  printf("Backward correctness: %s\n", (ok_dx && ok_dg && ok_db) ? "PASS" : "FAIL");

  // ---------------------------
  // Benchmark: forward/backward
  // ---------------------------
  int warmup = 20;
  int benchIters = iters;

  float f_ms = bench_kernel_ms(ln_forward_warp, f_grid, f_block, warmup, benchIters,
                               x, gamma, beta, y, mean, inv, B, D, eps);

  float b_ms = bench_kernel_ms(ln_backward_warp, b_grid, b_block, warmup, benchIters,
                               x, gamma, mean, inv, dout, dx, dgamma, dbeta, B, D);

  printf("Benchmark forward:  %.4f ms\n", f_ms);
  printf("Benchmark backward: %.4f ms\n", b_ms);

  // Cleanup
  CUDA_CHECK(cudaFree(x));
  CUDA_CHECK(cudaFree(gamma));
  CUDA_CHECK(cudaFree(beta));
  CUDA_CHECK(cudaFree(y));
  CUDA_CHECK(cudaFree(mean));
  CUDA_CHECK(cudaFree(inv));
  CUDA_CHECK(cudaFree(dout));
  CUDA_CHECK(cudaFree(dx));
  CUDA_CHECK(cudaFree(dgamma));
  CUDA_CHECK(cudaFree(dbeta));

  std::free(hx);
  std::free(hgamma);
  std::free(hbeta);
  std::free(hy_ref);
  std::free(hy_gpu);
  std::free(hmean_ref);
  std::free(hinv_ref);
  std::free(hmean_gpu);
  std::free(hinv_gpu);
  std::free(hdout);
  std::free(hdx_ref);
  std::free(hdx_gpu);
  std::free(hdg_ref);
  std::free(hdb_ref);
  std::free(hdg_gpu);
  std::free(hdb_gpu);

  return (ok_y && ok_m && ok_i && ok_dx && ok_dg && ok_db) ? 0 : 2;
}


Overwriting ln_warp_skeleton.cu


In [ ]:
!nvcc -arch=sm_75 ln_warp_skeleton.cu -o ln_warp_skeleton
!./ln_warp_skeleton

LN Skeleton: B=4096 D=1024 iters=200
Forward correctness: PASS
Backward correctness: PASS
Benchmark forward:  0.2118 ms
Benchmark backward: 0.2893 ms


In [ ]:
!ncu --set default --kernel-name "ln_forward_warp" -o ncu_ln_fwd ./ln_warp_skeleton 4096 1024 50
!ncu --set default --kernel-name "ln_backward_warp" -o ncu_ln_bwd ./ln_warp_skeleton 4096 1024 50

## 🧩 Additional Task — Accuracy Comparison: Mixed Precision (FP16/BF16 I/O, FP32 Reduction) vs. Pure Low Precision

### 🎯 Objective
Compare numerical accuracy between two LayerNorm implementations:

1) **Mixed precision (recommended in practice)**  
- **Input/Output:** FP16 or BF16  
- **Reductions (mean/variance + key accumulations):** FP32  
- **Output:** cast back to FP16/BF16  

2) **Pure low precision (no accuracy fine-tune)**  
- **Input/Output:** FP16 or BF16  
- **Reductions and accumulations:** also FP16/BF16 (or minimal FP32 usage)  
- No special stabilization beyond the baseline formula

The goal is to quantify how much accuracy you gain by doing **FP32 reductions** while keeping storage bandwidth low.

---

### 🧠 What You Should Implement / Compare
You should create two kernel variants (forward + backward optional):

#### Variant A — Mixed Precision (FP32 Reduction)
- Load `x`, `gamma`, `beta` as FP16/BF16
- Convert to FP32 in registers for:
  - sum / sumsq reductions
  - normalization math
  - key gradient reductions (if backward included)
- Compute `y` in FP32 then cast to FP16/BF16 and store

#### Variant B — Pure Low Precision (No Fine-tune)
- Load and compute in FP16/BF16 as much as possible
- Compute mean/variance in FP16/BF16 (or minimal conversion)
- Store output directly in FP16/BF16
- No additional tricks to improve stability

---

### 📊 Metrics to Report (Accuracy)
For forward:
- **Max absolute error**: `max |y - y_ref|`
- **Mean absolute error (MAE)**: `mean |y - y_ref|`
- **Relative error** (optional): `|y - y_ref| / (|y_ref| + eps)`
- **Row-wise stats drift** (optional):
  - mean error per row
  - variance error per row

For backward (optional but valuable):
- Error metrics for:
  - `dx`
  - `dgamma`
  - `dbeta`

Reference:
- Use **FP32 CPU reference** as the ground truth.

---

### 🧪 Experimental Setup Guidelines
Test multiple regimes to stress numerical stability:
- Different feature sizes `D`: e.g., 256 / 1024 / 4096
- Different batch sizes `B`: e.g., 256 / 4096
- Input distributions:
  - standard normal
  - wider variance (e.g., scaled normal)
  - adversarial-ish ranges (e.g., large magnitude values)

Run each configuration for both:
- **FP16**
- **BF16**

---

### ✅ Deliverables
1. Two LN kernel variants:
   - Mixed precision (FP32 reduction)
   - Pure low precision (no fine-tune)
2. An evaluation script/report that outputs:
   - error metrics for forward (and backward if included)
   - a short conclusion: when pure low precision breaks down
3. (Optional) Performance numbers:
   - runtime (ms)
   - throughput
   - trade-off: accuracy vs speed

---

### 🎓 Expected Outcome
You should be able to clearly demonstrate:

- **FP32 reduction dramatically improves stability**, especially for large `D`
- Pure FP16/BF16 reductions can introduce:
  - noticeable output drift
  - unstable variance estimates
  - larger gradient errors (if backward is included)

---

### 🧠 Key Takeaway
> **Storing activations in FP16/BF16 is bandwidth-efficient, but performing reductions in FP32 is often necessary to preserve numerical accuracy—especially in LayerNorm and other reduction-heavy operators.**


In [ ]:
%%writefile ln_mixed_precision_accuracy_skeleton.cu
#include <cstdio>
#include <cstdlib>
#include <cmath>
#include <cstring>
#include <cuda_runtime.h>

#if defined(__CUDA_ARCH__) || defined(__CUDACC__)
  #include <cuda_fp16.h>
#endif

#define CUDA_CHECK(call) do {                                   \
  cudaError_t err = (call);                                     \
  if (err != cudaSuccess) {                                     \
    fprintf(stderr, "CUDA error %s:%d: %s\n",                   \
            __FILE__, __LINE__, cudaGetErrorString(err));       \
    std::exit(EXIT_FAILURE);                                    \
  }                                                             \
} while(0)

// ------------------------------------------------------------
// Build-time dtype selection:
//   -DUSE_FP16=1   => use half for IO
//   -DUSE_BF16=1   => use __nv_bfloat16 for IO (requires CUDA bf16 support)
// Exactly one of USE_FP16 / USE_BF16 should be 1.
// ------------------------------------------------------------
#ifndef USE_FP16
#define USE_FP16 0
#endif
#ifndef USE_BF16
#define USE_BF16 0
#endif

#if (USE_FP16 + USE_BF16) != 1
#error "Define exactly one: -DUSE_FP16=1 or -DUSE_BF16=1"
#endif

#if USE_BF16
  #include <cuda_bf16.h>
  using half_t = __nv_bfloat16;
#else
  using half_t = half;
#endif

// ------------------------------------------------------------
// TODO: type conversion helpers
// Requirements:
//  - load half_t -> float
//  - store float -> half_t
// Note: for BF16 you can use __bfloat162float / __float2bfloat16
// ------------------------------------------------------------
__device__ __forceinline__ float to_float(half_t x) {
  // TODO
  return 0.0f;
}
__device__ __forceinline__ half_t from_float(float x) {
  // TODO
  return (half_t)0;
}

// ============================================================
// CPU reference in FP32 (ground truth)
// y = (x - mean) * inv_std * gamma + beta
// x, gamma, beta provided as float (already converted)
// ============================================================
static void ln_forward_cpu_f32(const float* x, const float* gamma, const float* beta,
                               float* y, float* mean, float* inv_std,
                               int B, int D, float eps) {
  for (int b = 0; b < B; ++b) {
    const float* xb = x + b * D;
    float m = 0.0f;
    for (int i = 0; i < D; ++i) m += xb[i];
    m /= (float)D;

    float v = 0.0f;
    for (int i = 0; i < D; ++i) {
      float d = xb[i] - m;
      v += d * d;
    }
    v /= (float)D;

    float inv = 1.0f / std::sqrt(v + eps);
    mean[b] = m;
    inv_std[b] = inv;

    float* yb = y + b * D;
    for (int i = 0; i < D; ++i) {
      float xhat = (xb[i] - m) * inv;
      yb[i] = xhat * gamma[i] + beta[i];
    }
  }
}

// ============================================================
// TODO: Warp reduction (sum) helper using __shfl_down_sync
// ============================================================
__device__ __forceinline__ float warpReduceSum(float v) {
  // TODO
  return v;
}

// ============================================================
// Variant A: Mixed precision IO + FP32 reduction + cast back
// - Input/Output: half_t
// - Reductions and core math: float
// - Store output: half_t
// Saves mean and inv_std as float for debugging/accuracy measurement.
// ============================================================
__global__ void ln_forward_mixed_fp32red(const half_t* __restrict__ x,
                                        const half_t* __restrict__ gamma,
                                        const half_t* __restrict__ beta,
                                        half_t* __restrict__ y,
                                        float* __restrict__ mean,
                                        float* __restrict__ inv_std,
                                        int B, int D, float eps) {
  // TODO:
  // - map one row per block/warp (your choice)
  // - accumulate sum and sumsq in FP32
  // - compute mean and inv_std in FP32
  // - normalize + affine in FP32
  // - cast output back to half_t
}

// ============================================================
// Variant B: Pure low precision (no accuracy fine-tune)
// - Input/Output: half_t
// - Reduction/accumulation: low precision as much as possible
// - No special stabilization beyond the basic formula
// Saves mean and inv_std as float for comparison (optional).
// ============================================================
__global__ void ln_forward_pure_lowp(const half_t* __restrict__ x,
                                    const half_t* __restrict__ gamma,
                                    const half_t* __restrict__ beta,
                                    half_t* __restrict__ y,
                                    float* __restrict__ mean,
                                    float* __restrict__ inv_std,
                                    int B, int D, float eps) {
  // TODO:
  // - do reductions in low precision (or minimally cast)
  // - compute mean/var in low precision path
  // - normalize + affine mostly in low precision
  // - cast/store output
}

// ============================================================
// Utilities: random init + conversions for CPU reference
// ============================================================
static inline float frand(unsigned int &s) {
  s = 1664525u * s + 1013904223u;
  return (s & 0x00FFFFFF) / float(0x01000000); // [0,1)
}

static void init_half_data(half_t* hx, half_t* hgamma, half_t* hbeta,
                           int B, int D, float scale_x) {
  unsigned int st = 123u;
  for (int i = 0; i < B * D; ++i) {
    float v = (frand(st) - 0.5f) * 2.0f * scale_x;
    // TODO: convert float -> half_t
    hx[i] = (half_t)0;
  }
  for (int i = 0; i < D; ++i) {
    float g = 0.5f + frand(st);
    float b = (frand(st) - 0.5f) * 0.1f;
    // TODO: convert float -> half_t
    hgamma[i] = (half_t)0;
    hbeta[i]  = (half_t)0;
  }
}

static void half_to_float_host(const half_t* in, float* out, int n) {
  for (int i = 0; i < n; ++i) {
    // TODO: convert half_t -> float (host-side)
    out[i] = 0.0f;
  }
}

// ============================================================
// Accuracy metrics
// ============================================================
struct ErrStats {
  double max_abs = 0.0;
  double mae = 0.0;
  double max_rel = 0.0;
};

static ErrStats compute_error(const float* y, const float* yref, int n, float eps) {
  ErrStats s;
  double sum_abs = 0.0;
  for (int i = 0; i < n; ++i) {
    double a = (double)y[i];
    double b = (double)yref[i];
    double diff = std::fabs(a - b);
    double rel = diff / (std::fabs(b) + eps);
    sum_abs += diff;
    if (diff > s.max_abs) s.max_abs = diff;
    if (rel  > s.max_rel) s.max_rel = rel;
  }
  s.mae = sum_abs / (double)n;
  return s;
}

// ============================================================
// Benchmark helper
// ============================================================
template <typename Kernel, typename... Args>
static float bench_kernel_ms(Kernel k, dim3 grid, dim3 block,
                             int warmup, int iters, Args... args) {
  for (int i = 0; i < warmup; ++i) {
    k<<<grid, block>>>(args...);
  }
  CUDA_CHECK(cudaGetLastError());
  CUDA_CHECK(cudaDeviceSynchronize());

  cudaEvent_t start, stop;
  CUDA_CHECK(cudaEventCreate(&start));
  CUDA_CHECK(cudaEventCreate(&stop));

  CUDA_CHECK(cudaEventRecord(start));
  for (int i = 0; i < iters; ++i) {
    k<<<grid, block>>>(args...);
  }
  CUDA_CHECK(cudaEventRecord(stop));
  CUDA_CHECK(cudaGetLastError());
  CUDA_CHECK(cudaEventSynchronize(stop));

  float ms = 0.0f;
  CUDA_CHECK(cudaEventElapsedTime(&ms, start, stop));
  CUDA_CHECK(cudaEventDestroy(start));
  CUDA_CHECK(cudaEventDestroy(stop));
  return ms / iters;
}

int main(int argc, char** argv) {
  // Args: ./ln_acc <B> <D> <iters> <scale_x>
  int B = (argc > 1) ? std::atoi(argv[1]) : 4096;
  int D = (argc > 2) ? std::atoi(argv[2]) : 1024;
  int iters = (argc > 3) ? std::atoi(argv[3]) : 200;
  float scale_x = (argc > 4) ? std::atof(argv[4]) : 1.0f;
  float eps = 1e-5f;

  printf("LN Accuracy Skeleton: B=%d D=%d iters=%d scale_x=%g\n", B, D, iters, scale_x);

  size_t bytes_x = (size_t)B * D * sizeof(half_t);
  size_t bytes_d = (size_t)D * sizeof(half_t);
  size_t bytes_y = (size_t)B * D * sizeof(half_t);
  size_t bytes_b = (size_t)B * sizeof(float);

  // Host (half)
  half_t* hx = (half_t*)std::malloc(bytes_x);
  half_t* hgamma = (half_t*)std::malloc(bytes_d);
  half_t* hbeta  = (half_t*)std::malloc(bytes_d);
  half_t* hy_mixed = (half_t*)std::malloc(bytes_y);
  half_t* hy_pure  = (half_t*)std::malloc(bytes_y);

  // Host (float ref + float views)
  float* hx_f32 = (float*)std::malloc((size_t)B * D * sizeof(float));
  float* hgamma_f32 = (float*)std::malloc((size_t)D * sizeof(float));
  float* hbeta_f32  = (float*)std::malloc((size_t)D * sizeof(float));
  float* hy_ref = (float*)std::malloc((size_t)B * D * sizeof(float));
  float* hmean_ref = (float*)std::malloc((size_t)B * sizeof(float));
  float* hinv_ref  = (float*)std::malloc((size_t)B * sizeof(float));

  float* hy_mixed_f32 = (float*)std::malloc((size_t)B * D * sizeof(float));
  float* hy_pure_f32  = (float*)std::malloc((size_t)B * D * sizeof(float));

  if (!hx||!hgamma||!hbeta||!hy_mixed||!hy_pure||!hx_f32||!hgamma_f32||!hbeta_f32||
      !hy_ref||!hmean_ref||!hinv_ref||!hy_mixed_f32||!hy_pure_f32) {
    fprintf(stderr, "Host malloc failed.\n");
    return 1;
  }

  // Init
  init_half_data(hx, hgamma, hbeta, B, D, scale_x);

  // Convert to float for CPU ref
  half_to_float_host(hx, hx_f32, B * D);
  half_to_float_host(hgamma, hgamma_f32, D);
  half_to_float_host(hbeta,  hbeta_f32,  D);

  // CPU reference FP32
  ln_forward_cpu_f32(hx_f32, hgamma_f32, hbeta_f32, hy_ref, hmean_ref, hinv_ref, B, D, eps);

  // Device alloc
  half_t *dx=nullptr, *dg=nullptr, *db=nullptr; // unused, placeholder
  half_t *x=nullptr, *gamma=nullptr, *beta=nullptr, *y_mixed=nullptr, *y_pure=nullptr;
  float *mean_mixed=nullptr, *inv_mixed=nullptr, *mean_pure=nullptr, *inv_pure=nullptr;

  CUDA_CHECK(cudaMalloc(&x, bytes_x));
  CUDA_CHECK(cudaMalloc(&gamma, bytes_d));
  CUDA_CHECK(cudaMalloc(&beta, bytes_d));
  CUDA_CHECK(cudaMalloc(&y_mixed, bytes_y));
  CUDA_CHECK(cudaMalloc(&y_pure,  bytes_y));

  CUDA_CHECK(cudaMalloc(&mean_mixed, bytes_b));
  CUDA_CHECK(cudaMalloc(&inv_mixed,  bytes_b));
  CUDA_CHECK(cudaMalloc(&mean_pure,  bytes_b));
  CUDA_CHECK(cudaMalloc(&inv_pure,   bytes_b));

  CUDA_CHECK(cudaMemcpy(x, hx, bytes_x, cudaMemcpyHostToDevice));
  CUDA_CHECK(cudaMemcpy(gamma, hgamma, bytes_d, cudaMemcpyHostToDevice));
  CUDA_CHECK(cudaMemcpy(beta,  hbeta,  bytes_d, cudaMemcpyHostToDevice));

  CUDA_CHECK(cudaMemset(y_mixed, 0, bytes_y));
  CUDA_CHECK(cudaMemset(y_pure,  0, bytes_y));
  CUDA_CHECK(cudaMemset(mean_mixed, 0, bytes_b));
  CUDA_CHECK(cudaMemset(inv_mixed,  0, bytes_b));
  CUDA_CHECK(cudaMemset(mean_pure,  0, bytes_b));
  CUDA_CHECK(cudaMemset(inv_pure,   0, bytes_b));

  // ------------------------------------------------------------
  // TODO: choose launch config (same for both variants)
  // ------------------------------------------------------------
  dim3 block(0,0,1); // TODO
  dim3 grid(0,0,1);  // TODO

  // Run both variants once
  ln_forward_mixed_fp32red<<<grid, block>>>(x, gamma, beta, y_mixed, mean_mixed, inv_mixed, B, D, eps);
  ln_forward_pure_lowp<<<grid, block>>>(x, gamma, beta, y_pure, mean_pure, inv_pure, B, D, eps);
  CUDA_CHECK(cudaGetLastError());
  CUDA_CHECK(cudaDeviceSynchronize());

  CUDA_CHECK(cudaMemcpy(hy_mixed, y_mixed, bytes_y, cudaMemcpyDeviceToHost));
  CUDA_CHECK(cudaMemcpy(hy_pure,  y_pure,  bytes_y, cudaMemcpyDeviceToHost));

  // Convert outputs to float for error metrics
  half_to_float_host(hy_mixed, hy_mixed_f32, B * D);
  half_to_float_host(hy_pure,  hy_pure_f32,  B * D);

  ErrStats e_mixed = compute_error(hy_mixed_f32, hy_ref, B*D, 1e-8f);
  ErrStats e_pure  = compute_error(hy_pure_f32,  hy_ref, B*D, 1e-8f);

  printf("Accuracy vs FP32 ref:\n");
  printf("  Mixed (FP32 reduction): max_abs=%.6g  mae=%.6g  max_rel=%.6g\n",
         e_mixed.max_abs, e_mixed.mae, e_mixed.max_rel);
  printf("  Pure  (lowp reduction): max_abs=%.6g  mae=%.6g  max_rel=%.6g\n",
         e_pure.max_abs, e_pure.mae, e_pure.max_rel);

  // Benchmark
  int warmup = 20;
  int benchIters = iters;

  float ms_mixed = bench_kernel_ms(ln_forward_mixed_fp32red, grid, block, warmup, benchIters,
                                   x, gamma, beta, y_mixed, mean_mixed, inv_mixed, B, D, eps);

  float ms_pure  = bench_kernel_ms(ln_forward_pure_lowp, grid, block, warmup, benchIters,
                                   x, gamma, beta, y_pure, mean_pure, inv_pure, B, D, eps);

  printf("Benchmark:\n");
  printf("  Mixed (FP32 reduction): %.4f ms\n", ms_mixed);
  printf("  Pure  (lowp reduction): %.4f ms\n", ms_pure);

  // Cleanup
  CUDA_CHECK(cudaFree(x));
  CUDA_CHECK(cudaFree(gamma));
  CUDA_CHECK(cudaFree(beta));
  CUDA_CHECK(cudaFree(y_mixed));
  CUDA_CHECK(cudaFree(y_pure));
  CUDA_CHECK(cudaFree(mean_mixed));
  CUDA_CHECK(cudaFree(inv_mixed));
  CUDA_CHECK(cudaFree(mean_pure));
  CUDA_CHECK(cudaFree(inv_pure));

  std::free(hx);
  std::free(hgamma);
  std::free(hbeta);
  std::free(hy_mixed);
  std::free(hy_pure);
  std::free(hx_f32);
  std::free(hgamma_f32);
  std::free(hbeta_f32);
  std::free(hy_ref);
  std::free(hmean_ref);
  std::free(hinv_ref);
  std::free(hy_mixed_f32);
  std::free(hy_pure_f32);

  return 0;
}


In [ ]:
!nvcc -arch=sm_75 ln_mixed_precision_accuracy_skeleton.cu -o ln_mixed_precision_accuracy_skeleton
!./ln_mixed_precision_accuracy_skeleton
